In [30]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import GlobalAveragePooling2D
#from tensorflow.keras.losses import BinaryCrossentropy

# Convolution with Batch Norm & ReLU

In [2]:
class ConvBNReLU(Model):   
    def __init__(self, filters, kernel_size, strides=1):
        super().__init__()
        # Conv2D 
        self.Conv = Conv2D(filters, kernel_size, strides)
        # Batch Normalization
        self.BN = BatchNormalization()
        # Activation Function
        self.AF = Activation("relu")
    def call(self, x):
        x = self.Conv(x)
        x = self.BN(x)
        return x = self.AF(x)

# Depthwise Separable Convolution

In [56]:
class depthwise_separable(Model):
    def __init__(self, filters, kernel_size):
        super().__init__()
        #add padding
        self.addPadding = ZeroPadding2D(padding=1)
        # Depthwise Convolution 3x3
        self.DWSeparableConv = SeparableConv2D(filters, kernel_size)
        # Batch Normalization
        self.BN = BatchNormalization()
        # Activation Function
        self.AF = Activation("relu")
    def call(self, x):
        x = self.addPadding(x)
        x = self.DWSeparableConv(x)
        x = self.BN(x)
        return self.AF(x)

# Separable Squeeze-Expand Block

In [55]:
class SSEModel(Model):
    def __init__(self, squeeze, expand):
        super().__init__()
        #squeeze
        self.pointwise1 = ConvBNReLU(squeeze, 1)
        #expand
        self.pointwise2 = ConvBNReLU(expand, 1)
        self.depthwise = depthwise_separable(expand, 3)
    def call(self, x):
        x = self.pointwise1(x)
        y = self.pointwise2(x)
        z = self.depthwise(x)
        return tf.keras.layers.Concatenate()([y, z])

# Slim Module

In [58]:
class SlimModule(Model):
    def __init__(self, squeeze, expand, dws, filtersNo):
        super().__init__()
        #SSE block 1
        self.SSE1 = SSEModel(squeeze, expand)
        #skip connection
        self.SkipConnection = 0
        #Dense
        self.dns = Dense(filtersNo)
        #SSE block 2
        self.SSE2 = SSEModel(squeeze, expand)
        
        #depthwise separable conv missing?
        self.depthwise = depthwise_separable(dws, 3)
    def call(self, x):
        s1 = self.SSE1(x)
        s1 = self.dns(s1)
        SkipConnection = Add()([x, s1])
        s2 = self.SSE2(SkipConnection)
        return self.depthwise(s2)

# Data

In [6]:
from tensorflow.keras.datasets import cifar10
batch_size = 32
num_classes = 10
epochs = 1

# Loads the CIFAR dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Display our data shape/dimensions
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Format our training data by Normalizing and changing data type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Now we one hot encode outputs
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


# SlimNet - CNN

In [64]:
inputShape = (178, 218, 3)
slim = Sequential()
#input = Input(shape=(178, 218, 3))

#slim.add(ConvBNReLU(96, 7, 2))
slim.add(Conv2D(96, (7, 7), (2, 2), padding='same',
                 input_shape=x_train.shape[1:]))
slim.add(Activation('relu'))

#Slim module 1
slim.add(SlimModule(16, 64, 48, 96))
slim.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
#Slim module 2
slim.add(SlimModule(32, 128, 96, 48))
slim.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
#Slim module 3
slim.add(SlimModule(48, 192, 144, 96))
slim.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
#Slim module 4
slim.add(SlimModule(64, 256, 192, 144))
slim.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))

slim.add(GlobalAveragePooling2D())

slim.add(Dense(40))

# initiate RMSprop optimizer and configure some parameters
opt = tf.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

slim.compile(loss = 'categorical_crossentropy',
              optimizer = opt,
              metrics = ['accuracy'])

print(slim.summary())
        

ValueError: Negative dimension size caused by subtracting 3 from 1 for 'max_pooling2d_15/MaxPool' (op: 'MaxPool') with input shapes: [?,1,1,192].